In [104]:
function ortho2eva(mmax, z, w)
    # Initialize polynomial output array
    pols = zeros(Float64, (mmax + 1) * (mmax + 2) ÷ 2)

    # Constants based on the geometry of the triangle
    c0 = 1/sqrt(3)*sqrt(sqrt(3))
    c1 = sqrt(2)*sqrt(sqrt(3))
    c2 = sqrt(2)*sqrt(sqrt(3))

    if mmax == 0
        pols[1] = c0
        return pols
    elseif mmax == 1
        pols[1] = c0
        pols[2] = z[1] * c1
        pols[3] = z[2] * c2
        return pols
    end

    # Memory management for work arrays
    iw1 = 1
    iw2 = iw1 + mmax + 1

    pols = ortho2eva0(mmax, z, pols, w[iw1:iw1+mmax], w[iw2:iw2+(mmax+1)^2-1])
    return pols
end

function ortho2eva0(mmax, z, pols, f1, f2)
    # Constants for coordinate transformation
    sqrt3 = sqrt(3.0)
    r11 = -1.0 / 3.0
    r12 = -1.0 / sqrt3
    r21 = 1.0 / 9.0 * (-sqrt3) * sqrt3
    r22 = 1.0 / 9.0 * (6.0) * sqrt3

    # Map the standard triangle to another coordinate system
    a, b = z
    x = r11 + r12 * b + a
    y = r21 + r22 * b

    # Evaluate Koornwinder's polynomials via three terms recursion
    par1 = (2 * x + 1 + y) / 2
    par2 = (1 - y) / 2
    f1 = klegeypols(par1, par2, mmax)

    # Loop to calculate polynomial values
    kk = 0
    for m in 0:mmax
        par1 = 2 * m + 1
        # Properly using view with an explicit end for the slice
        f2 = kjacopols(y, par1, 0, mmax - m)
        for n in 0:m
            kk += 1
            pols[kk] = f1[m - n + 1] * f2[n + 1 + (m - n) * (mmax + 1)]

            # Normalize
            scale = sqrt((1.0 + (m - n) + n) * (1.0 + (m - n) + (m - n)) / sqrt3)
            pols[kk] *= scale
        end
    end
    return pols
end


# Example usage
mmax = 3
z = [0.5, 0.5]
w = zeros(Float64, 100)  # Ensure this is large enough based on calculation needs
pols = ortho2eva(mmax, z, w)
println("Orthogonal polynomials evaluated at z: ", pols)


BoundsError: BoundsError: attempt to access 3-element Vector{Float64} at index [5]

In [98]:
function ortho2eva3(mmax, z, w)
    pols = zeros(Float64, (mmax + 1) * (mmax + 2) ÷ 2)  # Initialize polynomial values
    dersx = zeros(Float64, length(pols))  # Initialize derivative w.r.t x
    dersy = zeros(Float64, length(pols))  # Initialize derivative w.r.t y
    
    # Set up indices for the work arrays based on the assumed structure of w
    iw1 = 1
    iw2 = iw1 + mmax + 1
    iw3 = iw2 + (mmax + 1)^2
    iw4 = iw3 + mmax + 1
    iw5 = iw4 + mmax + 1
    iw6 = iw5 + (mmax + 1)^2

    ortho2eva30(mmax, z, pols, dersx, dersy, w[iw1:iw2-1], w[iw2:iw3-1], w[iw3:iw4-1], w[iw4:iw5-1], w[iw5:iw6-1], w[iw6:end])
    
    return pols, dersx, dersy
end

function ortho2eva30(mmax, z, pols, dersx, dersy, f1, f2, f3, f4, f5, f6)
    # Initialize constants
    sqrt3 = sqrt(3.0)
    r11 = -1.0 / 3.0
    r12 = -1.0 / sqrt3
    r21 = 1.0 / 9.0 * (-sqrt3) * sqrt3
    r22 = 1.0 / 9.0 * (6.0) * sqrt3

    # Mapping from standard triangle to another coordinate system
    x, y = z
    a = r11 + r12 * y + x
    b = r21 + r22 * y

    # Evaluate Koornwinder's polynomials using helper functions
    par1 = (2 * a + 1 + b) / 2
    par2 = (1 - b) / 2
    f1, f3, f4 = klegeypols3(par1, par2, mmax)

    # Nested loops to evaluate polynomials and their derivatives
    kk = 0
    for m in 0:mmax
        par1 = 2 * m + 1
        kjacopols2(b, par1, 0, mmax - m, f2, f5)
        for n in 0:m
            kk += 1
            pols[kk] = f1[m - n + 1] * f2[n + 1]
            dersx[kk] = f3[m - n + 1] * f2[n + 1]
            dersy[kk] = f1[m - n + 1] * f5[n + 1] * r22 +
                        f3[m - n + 1] * f2[n + 1] * (r12 + r22 / 2) +
                        f4[m - n + 1] * f2[n + 1] * (-r22 / 2)

            # Scale the results
            scale = sqrt((1.0 + (m - n) + n) * (1.0 + (m - n) + (m - n)) / sqrt3)
            pols[kk] *= scale
            dersx[kk] *= scale
            dersy[kk] *= scale
        end
    end
end

# Example usage
mmax = 5
z = [0.5, 0.5]
w = zeros(Float64, 90)  # Example size, should be adjusted based on actual calculation needs
pols, dersx, dersy = ortho2eva3(mmax, z, w)
println("Polynomials: ", pols)
println("Derivatives w.r.t x: ", dersx)
println("Derivatives w.r.t y: ", dersy)


Polynomials: [0.7598356856515927, 0.9306048591020997, 2.267388053052527, 0.8933292989915644, 3.822800893970047, 1.4435487784898635, 0.8256121381579707, 4.743043541965731, 2.2567288413204842, 0.813273350245399, 0.7500176197461961, 5.392616708346702, 1.9775843585249209, 0.7874485353536854, -1.0299015703500418, 0.6725998921671886, 4.799882060615391, 1.7338868962800298, 0.6761038908192225, -0.9770504184270808, 0.8228911534152307]
Derivatives w.r.t x: [0.0, 1.8612097182041993, 0.0, 4.414246434574068, 7.645601787940094, 0.0, 6.677064930081849, 23.43700477280494, 4.5134576826409685, 0.0, 8.418980043986231, 43.61230926789707, 9.7719225301828, 1.5748970707073708, -0.0, 9.639527806270781, 53.878882599429, 14.022656466400791, 3.3408611951040355, -1.9541008368541617, 0.0]
Derivatives w.r.t y: [0.0, -2.0663578828064015e-16, 3.1020161970069986, 0.6422243371676292, 4.606259045333721, 3.907726383351048, 1.3161682327315474, 8.769802381227711, 4.643440301204176, 9.545378472043451, 1.8472866174851739, 14

In [64]:
function klegeypols(x, y, n)
    # Initialize the array for storing polynomial values
    pols = zeros(Float64, n + 1)
    
    # Handle the case where n is 0 or 1
    if n >= 2
        pols[1] = 1.0
        pols[2] = x
    else
        pols[1] = 1.0
        if n == 0
            return pols
        end
        pols[2] = x
        return pols
    end

    # Recursive computation for n > 1
    pkm1 = 1.0
    pk = x

    for k in 1:n-1
        pkp1 = ((2 * k + 1) * x * pk - k * pkm1 * y^2) / (k + 1)
        pols[k + 2] = pkp1

        pkm1 = pk
        pk = pkp1
    end

    return pols
end

# Example usage
x = 0.5
y = 0.5
n = 5
pols = klegeypols(x, y, n)
println("Scaled Legendre Polynomials: ", pols)


Scaled Legendre Polynomials: [1.0, 0.5, 0.25, 0.125, 0.0625, 0.03125]


In [65]:
function klegeypols3(x, y, n)
    # Initialize arrays for polynomials and their derivatives with respect to x and y
    pols = zeros(Float64, n + 1)
    dersx = zeros(Float64, n + 1)
    dersy = zeros(Float64, n + 1)
    
    # Handle the base cases: n = 0 or n = 1
    if n >= 2
        pols[1] = 1.0
        dersx[1] = 0.0
        dersy[1] = 0.0
        pols[2] = x
        dersx[2] = 1.0
        dersy[2] = 0.0
    else
        pols[1] = 1.0
        dersx[1] = 0.0
        dersy[1] = 0.0
        if n == 0
            return pols, dersx, dersy
        end
        pols[2] = x
        dersx[2] = 1.0
        dersy[2] = 0.0
        return pols, dersx, dersy
    end

    # Recursion for n > 1
    pkm1 = 1.0
    pk = x
    dkm1 = 0.0
    dk = 1.0
    ykm1 = 0.0
    yk = 0.0
    
    for k in 1:n-1
        pkp1 = ((2*k + 1) * x * pk - k * pkm1 * y * y) / (k + 1)
        dkp1 = ((2*k + 1) * (x * dk + pk) - k * dkm1 * y * y) / (k + 1)
        ykp1 = ((2*k + 1) * (x * yk) - k * (pkm1 * 2 * y + ykm1 * y * y)) / (k + 1)
        
        pols[k + 2] = pkp1
        dersx[k + 2] = dkp1
        dersy[k + 2] = ykp1
        
        pkm1 = pk
        pk = pkp1
        dkm1 = dk
        dk = dkp1
        ykm1 = yk
        yk = ykp1
    end
    
    return pols, dersx, dersy
end

# Example usage
x = 0.5
y = 0.5
n = 8
pols, dersx, dersy = klegeypols3(x, y, n)
println("Polynomials: ", pols)
println("Derivatives w.r.t x: ", dersx)
println("Derivatives w.r.t y: ", dersy)


Polynomials: [1.0, 0.5, 0.25, 0.125, 0.0625, 0.03125, 0.015625, 0.0078125, 0.00390625]
Derivatives w.r.t x: [0.0, 1.0, 1.5, 1.5, 1.25, 0.9375, 0.65625, 0.4375, 0.28125]
Derivatives w.r.t y: [0.0, 0.0, -0.5, -0.75, -0.75, -0.625, -0.46875, -0.328125, -0.21875]


In [97]:
function kjacopols(x, a, b, n)
    # Initialize the array for polynomials
    pols = zeros(Float64, n + 1)
    
    # Handle the base cases: n = 0 or n = 1
    if n >= 2
        # Continue to the recursive case
        pols[1] = 1.0
        pols[2] = (a/2 - b/2) + (1 + a/2 + b/2) * x
    else
        pols[1] = 1.0
        if n == 0
            return pols
        end
        pols[2] = (a/2 - b/2) + (1 + a/2 + b/2) * x
        return pols
    end

    # Recursive computation for n >= 2
    pkm1 = 1.0
    pk = 1.0  # This should maintain the last calculated value instead of resetting to 1
    pkp1 = (a/2 - b/2) + (1 + a/2 + b/2) * x
    
    for k in 2:n
        # Calculate the next polynomial value using the recursion formula
        alpha = (2*k + a + b - 1) * (a^2 - b^2 + (2*k + a + b - 2) * (2*k + a + b) * x)
        beta = 2 * (k + a - 1) * (k + b - 1) * (2*k + a + b)
        pkp1 = (alpha * pk - beta * pkm1) / (2 * k * (k + a + b) * (2*k + a + b - 2))
        
        # Update the polynomial values
        pkm1 = pk
        pk = pkp1
        pols[k + 1] = pkp1
    end
    
    return pols
end

# Example usage
x = 0.5
a = 0.0
b = 0.0
n = 5
pols = kjacopols(x, a, b, n)
println("Polynomials: ", pols)


Polynomials: [1.0, 0.5, 0.25, -0.4583333333333333, -0.5885416666666666, -0.16302083333333336]


In [67]:
function kjacopols2(x, a, b, n)
    # Initialize arrays for the polynomials and their derivatives
    pols = zeros(Float64, n + 1)
    ders = zeros(Float64, n + 1)
    
    # Base cases: n = 0 or n = 1
    if n >= 2
        # Continue to the recursive case
        pols[1] = 1.0
        ders[1] = 0.0
        pols[2] = (a/2 - b/2) + (1 + a/2 + b/2) * x
        ders[2] = (1 + a/2 + b/2)
    else
        pols[1] = 1.0
        ders[1] = 0.0
        if n == 0
            return pols, ders
        end
        pols[2] = (a/2 - b/2) + (1 + a/2 + b/2) * x
        ders[2] = (1 + a/2 + b/2)
        return pols, ders
    end

    # Recursive computation for n >= 2
    pkm1 = 1.0
    dkm1 = 0.0
    pk = (a/2 - b/2) + (1 + a/2 + b/2) * x
    dk = (1 + a/2 + b/2)
    
    pk = 1.0  # Resetting pk to 1 seems to be a mistake in the Fortran code; possibly it should maintain its calculated value
    dk = 0.0  # Resetting dk to 0 for the recursion starting point
    
    for k in 2:n
        pkp1 = (a/2 - b/2) + (1 + a/2 + b/2) * x  # Calculate the next polynomial value
        dkp1 = (1 + a/2 + b/2)                    # Calculate the next derivative value
        
        alpha1 = (2*k + a + b - 1) * (a^2 - b^2)
        alpha2 = (2*k + a + b - 1) * ((2*k + a + b - 2) * (2*k + a + b))
        beta = 2 * (k + a - 1) * (k + b - 1) * (2*k + a + b)
        gamma = 2 * k * (k + a + b) * (2*k + a + b - 2)
        
        # Update the next polynomial and derivative values using the recurrence relation
        pkp1 = ((alpha1 + alpha2 * x) * pk - beta * pkm1) / gamma
        dkp1 = ((alpha1 + alpha2 * x) * dk - beta * dkm1 + alpha2 * pk) / gamma
        
        pkm1 = pk
        pk = pkp1
        dkm1 = dk
        dk = dkp1
        
        pols[k + 1] = pkp1
        ders[k + 1] = dkp1
    end
    
    return pols, ders
end

# Example usage
x = 0.5
a = 0.0
b = 0.0
n = 6
pols, ders = kjacopols2(x, a, b, n)
println("Polynomials: ", pols)
println("Derivatives: ", ders)


Polynomials: [1.0, 0.5, 0.25, -0.4583333333333333, -0.5885416666666666, -0.16302083333333336, 0.341015625]
Derivatives: [0.0, 1.0, 1.5, 1.6666666666666667, -0.46875, -2.8145833333333337, -2.4882812500000004]
